This notebook is to exemplify the obtention one of power law of a single treshold of the 2-point correlation function of enstrophy excursion sets. The objective is to provide an interactive enviroment to test different thresholds and provide a feel of what is the behavior of the system. 

In [1]:
#!ipcluster start -n 8 --engines=MPI --profile='mpi' # for parallel run: start the engines using terminal
from ipyparallel import Client
rc = Client(profile='mpi')

Following suit, it is necessary to import the libraries that are necessary to the work to be executed. These include  numpy for the generay array manipulations, pyFFTW to the Fourier Transforms, mpi4py for MPI support, general math, sys and os libraries and pyJHTDB, that provides a python wraper for the C-SOAP interface of the Johs Hopkins Turbulence Databases. We also import some modules that provide basic 3D FFT and histograming functionality build upon numpy and pyFFTW, as a way to streamline the overall process. Those modules, which include FFT3Dfield, IFFT3Dfield and EnergySpectrum, were originaly developed by Dr. Kun Yang.

In [2]:
%%px
# Import the libraries

import os
import sys
import math
import numpy as np
import pyfftw as ft 
from mpi4py import MPI
import matplotlib
import matplotlib.pyplot as plt
import pyJHTDB
from pyJHTDB.dbinfo import isotropic1024coarse
from pyJHTDB import libJHTDB

from fft3d import FFT3Dfield_new
from EnergySpectrum import EnergySpectrum

We first initialize general domain constants, based on the isotropic1024coarse dictionary, which provide information on the isotropic turbulence database. We then initialize MPI variables, including the communicator, rank and number of process. The typical number of processes is 8, both because it doesn't, usually, overloads the sciserver hardware, but also it is the optimal number of processes to query the database. Afterwards, it is necessary to initalize the variables that designate the domain associated with each MPI process. Due to the way that the FFT3D modules were build, we chose to divide the domain in slabs split in the X direction. Also, constants related to wavenumber and chunck size downloads are initialized.

In [3]:
%%px

Nx = isotropic1024coarse['nx']; Ny = isotropic1024coarse['ny']; Nz = isotropic1024coarse['nz']
Lx = isotropic1024coarse['lx']; Ly = isotropic1024coarse['ly']; Lz = isotropic1024coarse['lz']

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
nproc = comm.Get_size()
if(rank==0):
    print("n_proc = "+str(nproc))
    print("rank = "+str(rank))

# Computational Domain

nx=Nx//nproc; ny=Ny; nz=Nz
nz_half=nz//2
nek=int(math.sqrt(2.0)/3*Nx)
time = 0.0

chkSz = 32
slabs = nx//chkSz

[stdout:2] 
n_proc = 8
rank = 0


Instead of downloading the velocity field from the website, we just load the cached value of enstrophy pre-computed from a previous notebook. On vm01 it takes between 60 to 90 seconds to load from disk.

In [4]:
%%px

from DataDownload import DataDownload

# Populate velocity field from the Database

if(rank==0):
    print("Starting the loading process")

##########################################
load_from_file = True
store_on_file = False
auth_token = "com.gmail.jhelsas-b854269a"

folder = "/home/idies/workspace/scratch"
filename = "ref-isotropic1024coarse-"+str(rank)+"-(t="+str(time)+")"+".npz"
file = folder + "/" + filename

if(load_from_file):
    vx = ft.zeros_aligned((nx,ny,nz), dtype='float32')
    vy = ft.zeros_aligned((nx,ny,nz), dtype='float32')
    vz = ft.zeros_aligned((nx,ny,nz), dtype='float32')

    comm.Barrier(); t1=MPI.Wtime()
    content = np.load(file)
    if(int(content['nproc'])!=nproc):
        print("Unmatched number of processes. Must first pre-process to adequate number of process")
    vx = content['vx']
    vy = content['vy']
    vz = content['vz']
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load from disk: {0:.2f} seconds\n'.format(t2-t1))
else:
    comm.Barrier(); t1=MPI.Wtime()
    
    ddwnld = DataDownload()
    vx,vy,vz = ddwnld.DownldData_pyJHTDB('isotropic1024coarse',time,nx,ny,nz,nproc,rank,auth_token)
    
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        print("Finished loading")
        sys.stdout.write('Load field from database: {0:.2f} seconds\n'.format(t2-t1))
    
    comm.Barrier(); t1=MPI.Wtime()
    
    np.savez(file,vx=vx,vy=vy,vz=vz,nproc=nproc)
    comm.Barrier(); t2=MPI.Wtime()
    if(rank==0):
        sys.stdout.write('Caching the data: {0:.2f} seconds\n'.format(t2-t1))


if(rank==0):
    print("vx shape = "+str(vx.shape))       

[stdout:2] 
Starting the loading process
Finished loading
Load from disk: 21.38 seconds
vx shape = (128, 1024, 1024)


Here we need to alocate all arrays we need to do the calculations of the 2-point correlation function. Since there are many arrays to be alocated at the same time, it takes a fair amount of time. On vm01, it takes around 160 to 190 seconds. On vm04, it is taking 26 seconds.

In [5]:
%%px

dx = isotropic1024coarse['dx']
ner = int(1024*np.sqrt(3))
rbins = np.linspace(-0.5*dx,2*np.pi*np.sqrt(3)+0.5*dx,ner+1)

comm.Barrier(); t1=MPI.Wtime()

fft = FFT3Dfield_new(nx,ny,nz,nproc,rank)

X = np.zeros((nx,ny,nz), dtype='float32')
Y = np.zeros((nx,ny,nz), dtype='float32')
Z = np.zeros((nx,ny,nz), dtype='float32')

r2 = np.zeros((nx,ny,nz), dtype='float32')

chi = ft.zeros_aligned((nx,ny,nz), dtype='float32')
chi2 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
cchi = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')
cchi2 = ft.zeros_aligned((nx,ny,1+(nz//2)), dtype='complex64')
corr = ft.zeros_aligned((nx,ny,nz),dtype='float32')
iCorr = np.zeros((nx,ny,nz), dtype='float32')

R11 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
R12 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
R13 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
R21 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
R22 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
R23 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
R31 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
R32 = ft.zeros_aligned((nx,ny,nz), dtype='float32')
R33 = ft.zeros_aligned((nx,ny,nz), dtype='float32')

corrSum=np.zeros(ner,dtype='float32')
corrF=np.zeros(ner,dtype='float32')  

r2Sum=np.zeros(ner,dtype='float32')
r2F=np.zeros(ner,dtype='float32') 

comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Alocating vectors: {0:.2f} seconds\n'.format(t2-t1))

[stdout:2] Alocating vectors: 124.47 seconds


To properly arrange the real space radius to do the spherical integration, we need to assign the distance from the first coordinate, so we compute X, Y and Z arrays, with the respective coordinates, and from there we compute the distance squared to the origin. This is so because the relevant information is contained in the first octant, so all we need is to integrate spherically the first octant. This section takes a fair share of time, around 200-220 seconds. On vm04, this is taking around 23 seconds. 

In [6]:
%%px

comm.Barrier(); t1=MPI.Wtime()
for i in range(nx):
    if (i+rank*nx < Nx//2):
        X[i,:,:] = (i+rank*nx)*isotropic1024coarse['dx']
    else:
        X[i,:,:] = Lx-(i+rank*nx)*isotropic1024coarse['dx']

for j in range(ny):
    if (j < Ny//2):
        Y[:,j,:] = j*isotropic1024coarse['dy']
    else:
        Y[:,j,:] = Ly-j*isotropic1024coarse['dy']
    
for k in range(nz):
    if (k < Nz//2):
        Z[:,:,k] = k*isotropic1024coarse['dz']
    else:
        Z[:,:,k] = Lz-k*isotropic1024coarse['dz']
    
r2[:,:,:] = X[:,:,:]**2+Y[:,:,:]**2+Z[:,:,:]**2

r2rt = np.sqrt(r2)
comm.Barrier(); t2=MPI.Wtime()
if(rank==0):
    sys.stdout.write('Preparing the real domain for radial integration: {0:.2f} seconds\n'.format(t2-t1))

[stdout:2] Preparing the real domain for radial integration: 48.30 seconds


The computation of the full radial 2-point correlation function is done here, including the fourier transform and radial integration. The full computation takes around 80 seconds, on VM04. The breakdown time measured is:

Creating the excursion set: 1.18 seconds

Computing the 2-point correlation function: 48.87 seconds

Cleaning all but the 1st octant: 2.15 seconds

Integrating radially on each slab: 29.60 seconds

Adding the results form different slabs: 0.00 seconds

Full radial 2-point correlation calculation: 81.80 seconds

In [8]:
%%px

u2 = vx**2 + vy**2 + vz**2

avgu2 = np.average(u2)
avgu2Gl=np.zeros(1,dtype='float32')

comm.Reduce([avgu2,MPI.REAL],[avgu2Gl,MPI.REAL],op=MPI.SUM)
avgu2 = avgu2Gl[0]/nproc
avgu2 = comm.bcast(avgu2, root=0)

In [10]:
%%px
avgu2 = 0.5*avgu2
print(avgu2)

[stdout:0] 0.682719171047
[stdout:1] 0.682719171047
[stdout:2] 0.682719171047
[stdout:3] 0.682719171047
[stdout:4] 0.682719171047
[stdout:5] 0.682719171047
[stdout:6] 0.682719171047
[stdout:7] 0.682719171047


In [18]:
%%px 

hist = []
r2st = []
    
#######################################

comm.Barrier(); t1=MPI.Wtime()

chi[:,:,:] = vx[:,:,:]
chi2[:,:,:] = vx[:,:,:]

#######################################

comm.Barrier(); t2=MPI.Wtime()
cchi[:,:,:] = fft.forward3Dfft(chi,nx,ny,nz,nproc,rank)
cchi2[:,:,:] = fft.forward3Dfft(chi2,nx,ny,nz,nproc,rank)
    
tmp = cchi*(cchi2.conj())
corr[:,:,:] = fft.backward3Dfft(tmp,nx,ny,nz,nproc,rank)
    
#######################################
    
comm.Barrier(); t3=MPI.Wtime()
        
#######################################
    
comm.Barrier(); t4=MPI.Wtime()
    
corrLoc,redges = np.histogram(r2rt,range=(0.5*dx,(ner+0.5)*dx),bins = rbins,weights=corr)
    
r2Loc,r2edges = np.histogram(r2rt,range=(0.5*dx,(ner+0.5)*dx),bins = rbins)
       
#######################################
    
comm.Barrier(); t5=MPI.Wtime()
    
corrLoc=np.float32(corrLoc)
comm.Reduce([corrLoc,MPI.REAL],[corrSum,MPI.REAL],op=MPI.SUM)
np.copyto(corrF,corrSum)
    
corrApend=np.zeros(ner,dtype='float32') 
corrApend[:] = corrF[:]
    
#########################
    
r2Loc=np.float32(r2Loc)
comm.Reduce([r2Loc,MPI.REAL],[r2Sum,MPI.REAL],op=MPI.SUM)
np.copyto(r2F,r2Sum)
    
r2Apend=np.zeros(ner,dtype='float32') 
r2Apend[:] = r2F[:]
    
if rank==0:
    hist.append(corrApend[:])
    r2st.append(r2Apend[:])
    
#######################################
    
comm.Barrier(); t6=MPI.Wtime()
    
if rank==0:
    sys.stdout.write('Creating the excursion set: {0:.2f} seconds\n'.format(t2-t1))
    sys.stdout.write('Computing the 2-point correlation function: {0:.2f} seconds\n'.format(t3-t2))
    sys.stdout.write('Cleaning all but the 1st octant: {0:.2f} seconds\n'.format(t4-t3))
    sys.stdout.write('Integrating radially on each slab: {0:.2f} seconds\n'.format(t5-t4))
    sys.stdout.write('Adding the results form different slabs: {0:.2f} seconds\n'.format(t6-t5))
    sys.stdout.write('Full radial 2-point correlation calculation: {0:.2f} seconds\n'.format(t6-t1))
    sys.stdout.write('\n')

[stdout:6] 
Creating the excursion set: 10.12 seconds
Computing the 2-point correlation function: 74.63 seconds
Cleaning all but the 1st octant: 3.23 seconds
Integrating radially on each slab: 32.52 seconds
Adding the results form different slabs: 0.04 seconds
Full radial 2-point correlation calculation: 120.55 seconds

